* This notebook is demonstration of creating new feature on the basis of longitude and latitude.
* Feature engineering on the basis of aggregation.
* Target Encoding
* Feature Selection
* Stacking

In [1]:
# @title 連接google drive
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi
import os
os.chdir('/content/drive/My Drive')
os.getcwd()
os.chdir('/content/drive/My Drive/112 Sesmester/data2')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 1: nvidia-smi: command not found


'/content/drive/My Drive/112 Sesmester/data2'

## IMPORTING

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.impute import SimpleImputer,MissingIndicator, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate,GridSearchCV,RepeatedStratifiedKFold,cross_val_score,cross_val_predict
from sklearn.preprocessing import PowerTransformer,OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer,make_column_selector,ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score
from sklearn.impute import  KNNImputer
from sklearn.svm import SVC
from lightgbm import LGBMRegressor, LGBMClassifier
import xgboost as xgb

import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
#from kuma_utils.preprocessing.imputer import LGBMImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import holidays
import gc
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import PolynomialFeatures
from geopy.distance import geodesic

import gc

!pip install category_encoders
!pip install feature_engine #feature_engine.encoding feature_engine.discretisation feature_engine.encoding feature_engine.selection feature_engine.creation
from category_encoders import OrdinalEncoder


import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import KBinsDiscretizer,LabelEncoder
from feature_engine.encoding import OrdinalEncoder
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.encoding import RareLabelEncoder
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from feature_engine.selection import (RecursiveFeatureElimination, RecursiveFeatureAddition,
    DropConstantFeatures,
    DropDuplicateFeatures,
    SmartCorrelatedSelection,DropCorrelatedFeatures
)
from feature_engine.creation import MathFeatures,RelativeFeatures

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import MeanEncoder
from sklearn import metrics
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.cluster import KMeans
from category_encoders import MEstimateEncoder,OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import KBinsDiscretizer


from sklearn.ensemble import StackingRegressor


In [3]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor ,AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, ElasticNet, LassoLars, PassiveAggressiveRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
!pip install catboost
from catboost import CatBoostRegressor, metrics, Pool, cv
pd.set_option('display.max_columns', None)

## DATA LOADING

In [4]:

df = pd.read_csv("IsGreenOrder.csv")
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
# train = pd.read_csv("../input/food-delivery-dataset/train.csv")
# test = pd.read_csv("../input/food-delivery-dataset/test.csv")

## DATA CLEANING

In [6]:
#test=test.replace(" ","")
test=test.replace('NaN', float(np.nan), regex=True)

#train=train.replace(" ","")
train=train.replace('NaN', float(np.nan), regex=True)

In [7]:
train['Weatherconditions']=train['Weatherconditions'].str.split(" ", expand=True)[1]
test['Weatherconditions']=test['Weatherconditions'].str.split(" ", expand=True)[1]

train['Time_taken(min)']=train['Time_taken(min)'].str.split(" ", expand=True)[1]

In [8]:
num_cols = ['Delivery_person_Age','Delivery_person_Ratings','Restaurant_latitude','Restaurant_longitude',
            'Delivery_location_latitude','Delivery_location_longitude','Vehicle_condition',
            'multiple_deliveries','Time_taken(min)']
for col in num_cols:
    train[col]=train[col].astype('float64')

for col in num_cols[:-1]:
    test[col]=test[col].astype('float64')

train['Order_Date']=pd.to_datetime(train['Order_Date'],format="%Y-%m-%d")
test['Order_Date']=pd.to_datetime(test['Order_Date'],format="%Y-%m-%d")

In [9]:
train['Time_Orderd']=pd.to_timedelta(train['Time_Orderd'])
train['Time_Order_picked']=pd.to_timedelta(train['Time_Order_picked'])

train['Time_Order_picked_formatted']=np.where(train['Time_Order_picked'] < train['Time_Orderd'], train['Order_Date'] + pd.DateOffset(1)+train['Time_Order_picked'], train['Order_Date']+train['Time_Order_picked'])
train['Time_Ordered_formatted'] = train['Order_Date']+ train['Time_Orderd']
train['order_prepare_time_diff_mins']=((train['Time_Order_picked_formatted']- train['Time_Ordered_formatted']).dt.total_seconds())/60

In [10]:
test['Time_Orderd']=pd.to_timedelta(test['Time_Orderd'])
test['Time_Order_picked']=pd.to_timedelta(test['Time_Order_picked'])

test['Time_Order_picked_formatted']=np.where(test['Time_Order_picked'] < test['Time_Orderd'], test['Order_Date'] + pd.DateOffset(1)+test['Time_Order_picked'], test['Order_Date']+test['Time_Order_picked'])
test['Time_Ordered_formatted'] = test['Order_Date']+ test['Time_Orderd']
test['order_prepare_time_diff_mins']=((test['Time_Order_picked_formatted']- test['Time_Ordered_formatted']).dt.total_seconds())/60

In [11]:
cols=['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']
for col in cols:
    train[col]= abs(train[col])
for col in cols:
    test[col]= abs(test[col])

## Creating new features on the basis of latitude and longitude

In [12]:
train['distance_diff_KM']=np.zeros(len(train))
restaurant_cordinates_train=train[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
delivery_location_cordinates_train=train[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()

for i in range(len(train)):
    train['distance_diff_KM'].loc[i]=geodesic(restaurant_cordinates_train[i],delivery_location_cordinates_train[i])


In [13]:
test['distance_diff_KM']=np.zeros(len(test))
restaurant_cordinates_test=test[['Restaurant_latitude','Restaurant_longitude']].to_numpy()
delivery_location_cordinates_test=test[['Delivery_location_latitude','Delivery_location_longitude']].to_numpy()

for i in range(len(test)):
    test['distance_diff_KM'].loc[i]=geodesic(restaurant_cordinates_test[i],delivery_location_cordinates_test[i])


In [14]:

y=train["Time_taken(min)"]
train = train.drop(columns=["Time_taken(min)",'ID'],axis=1)
Id= test['ID'].str.strip()
test = test.drop(columns=["ID"],axis=1)


In [15]:
data=pd.concat([train,test]).reset_index(drop=True)

In [16]:
data.head()

,Unnamed: 0,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Distance_km,Route,Time_Orderd_in_minutes,location_cluster,date_category,IsGreenOrder,Time_Order_picked_formatted,Time_Ordered_formatted,order_prepare_time_diff_mins,distance_diff_KM,Time_taken(min)
0,26245,MUMRES14DEL02,28.0,4.9,19.181300,72.836191,19.311300,72.966191,2022-03-20,0 days 23:45:00,0 days 23:55:00,Sandstorms,Low,1.0,Snack,scooter,1.0,No,Metropolitian,41.1282,"[[19.1813, 72.83617], [19.18112, 72.83621], [1...",1425,3,27,1,2022-03-20 23:55:00,2022-03-20 23:45:00,10.0,11.737833961110447 km,NaN
1,15546,HYDRES17DEL01,23.0,4.2,17.451976,78.385883,17.561976,78.495883,2022-03-16,0 days 19:30:00,0 days 19:35:00,Fog,Jam,1.0,Snack,motorcycle,1.0,No,Metropolitian,27.8650,"[[17.45198, 78.38588], [17.45242, 78.38592], [...",1170,9,23,0,2022-03-16 19:35:00,2022-03-16 19:30:00,5.0,3.0205869101906617 km,NaN
2,130,SURRES03DEL01,29.0,4.6,21.186884,72.793616,21.226884,72.833616,2022-03-03,0 days 14:50:00,0 days 15:00:00,Sandstorms,High,1.0,Snack,scooter,0.0,No,Urban,7.4029,"[[21.1868, 72.79377], [21.18713, 72.79399], [2...",890,11,10,1,2022-03-03 15:00:00,2022-03-03 14:50:00,10.0,6.196979616064036 km,NaN
3,36780,INDORES09DEL02,27.0,4.1,22.725835,75.887648,22.855835,76.017648,2022-03-10,0 days 20:40:00,0 days 20:55:00,Stormy,Jam,0.0,Meal,motorcycle,1.0,No,Urban,26.2906,"[[22.72586, 75.88757], [22.72565, 75.88749], [...",1240,1,17,0,2022-03-10 20:55:00,2022-03-10 20:40:00,15.0,0.0,NaN
4,38179,LUDHRES18DEL03,20.0,4.7,30.890184,75.829615,30.920184,75.859615,2022-02-13,0 days 18:00:00,0 days 18:05:00,Sunny,Medium,2.0,Buffet,motorcycle,0.0,No,Urban,6.0020,"[[30.89024, 75.82966], [30.89037, 75.82946], [...",1080,13,2,0,2022-02-13 18:05:00,2022-02-13 18:00:00,5.0,0.0,NaN


In [17]:
na_cols=[]
for col in data.columns:
    if data[col].isna().sum()>0:
        na_cols.append(col)

data["n_missing"]= data[na_cols].isna().sum(axis=1)
data_missing_tag_df = data[na_cols].isna().astype(np.int8)
data_missing_tag_df.columns = [f"{c}_missing" for c in data_missing_tag_df.columns]

data=pd.concat([data, data_missing_tag_df], axis=1)

## Creating time based features

In [18]:
data["day"] = data.Order_Date.dt.day
data["week"] = data.Order_Date.dt.isocalendar().week
data["month"] = data.Order_Date.dt.month
data["quarter"] = data.Order_Date.dt.quarter
data["year"] = data.Order_Date.dt.year
data["hour"] = data.Time_Orderd.dt.components['hours']
data["dayofyear"] = data.Order_Date.dt.dayofyear
data['day_of_week'] = data.Order_Date.dt.day_of_week.astype(int)
data["is_month_start"] = data.Order_Date.dt.is_month_start.astype(int)
data["is_month_end"] = data.Order_Date.dt.is_month_end.astype(int)
data["is_quarter_start"] = data.Order_Date.dt.is_quarter_start.astype(int)
data["is_quarter_end"] = data.Order_Date.dt.is_quarter_end.astype(int)
data["is_year_start"] = data.Order_Date.dt.is_year_start.astype(int)
data["is_year_end"] = data.Order_Date.dt.is_year_end.astype(int)
data["is_leap_year"] = data.Order_Date.dt.is_leap_year.astype(int)
data["days_in_month"] = data.Order_Date.dt.days_in_month
data['is_weekend'] = np.where(data['day_of_week'].isin([5,6]),1,0)

In [19]:
data['distance_diff_KM']=data['distance_diff_KM'].astype("str").str.extract('(\d+)')
data['distance_diff_KM']=data['distance_diff_KM'].astype("int64")

In [20]:
data=data.drop(columns=['Order_Date','Time_Orderd','Time_Order_picked'],axis=1)
data=data.drop(columns=['Time_Order_picked_formatted','Time_Ordered_formatted'])

In [21]:
data['week']=data['week'].astype("int64")

In [22]:
for col in data.columns:
    if data[col].dtype == 'uint8':
        data[col]=data[col].astype("int64")

In [23]:
data['city_code']=data['Delivery_person_ID'].str.split("RES", expand=True)[0]

In [24]:
data=data.drop(columns=["Delivery_person_ID" ],  axis=1)

## Removing constant and duplicate features

In [25]:
pipe = Pipeline([
   ('constant', DropConstantFeatures(tol=0.998,missing_values='ignore')),
   ('duplicated', DropDuplicateFeatures(missing_values='ignore')),
])
data=pipe.fit_transform(data)


## More Feature Engineering

In [26]:
num_ft = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
mf=MathFeatures(['Delivery_person_Age','Delivery_person_Ratings'],missing_values='ignore',func= ["std",'mean'])
data=mf.fit_transform(data)

In [27]:
feat1=data.groupby("Delivery_person_Age")["Delivery_person_Ratings"].transform("mean")
data['feat2']=data['Delivery_person_Ratings'] - feat1

feat3=data.groupby("Delivery_person_Age")["Delivery_person_Ratings"].transform("std")
data['feat4']=data['Delivery_person_Ratings'] - feat3

data['feat9']=data.groupby(["Weatherconditions",'Road_traffic_density'])["Delivery_person_Age"].transform("mean")
#data['feat10']=data['Delivery_person_Age'] - data['feat9']

feat11=data.groupby(["Weatherconditions",'Road_traffic_density'])["Delivery_person_Age"].transform("std")
data['feat12']=data['Delivery_person_Age'] - feat11

data['feat13']=data.groupby(["Weatherconditions",'Road_traffic_density','City','city_code'])["distance_diff_KM"].transform("std")
data['feat14']=data['Delivery_person_Age'] - data['feat13']



In [28]:
data.to_csv('/content/drive/My Drive/112 Sesmester/data2/featureS.csv')
data.columns
columns_to_drop = ['Unnamed: 0', 'Time_Orderd_in_minutes','Time_taken(min)',
                   'Route', 'Time_taken(min)','City','multiple_deliveries','Type_of_vehicle']
data = data.drop(columns_to_drop, axis=1)
data.columns
# data['IsGreenOrder']

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weatherconditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Festival', 'Distance_km', 'location_cluster', 'date_category',
       'IsGreenOrder', 'order_prepare_time_diff_mins', 'distance_diff_KM',
       'n_missing', 'Delivery_person_Age_missing',
       'Delivery_person_Ratings_missing', 'multiple_deliveries_missing',
       'Festival_missing', 'City_missing', 'Time_taken(min)_missing', 'day',
       'week', 'month', 'quarter', 'hour', 'dayofyear', 'day_of_week',
       'is_month_start', 'is_month_end', 'is_quarter_start', 'days_in_month',
       'is_weekend', 'city_code',
       'std_Delivery_person_Age_Delivery_person_Ratings',
       'mean_Delivery_person_Age_Delivery_person_Ratings', 'feat2', 'feat4',
       'feat9', 'feat12', 'feat13', 'feat14'],
      dtype='object

In [29]:
# @title 填補缺失值
# Analyzing the dataset to suggest suitable methods for imputing missing values in each column
data=df
# Checking for missing values in each column
# Analyzing the dataset to suggest suitable methods for imputing missing values in each column

# Checking for missing values in each column
missing_values = data.isnull().sum()
missing_values = missing_values[missing_values > 0]

# Checking the data types of columns with missing values
data_types = data[missing_values.index].dtypes

# Preparing a summary of columns with missing values, their data types, and suggested imputation methods
imputation_methods = []

for column, missing_count in missing_values.iteritems():
    dtype = data_types[column]
    if dtype == 'object':
        # Suggesting mode imputation for categorical data
        method = 'Mode (for categorical data)'
    elif dtype in ['int64', 'float64']:
        # Suggesting mean or median imputation for numerical data, depending on the distribution
        if data[column].skew() < 1 and data[column].skew() > -1:
            method = 'Mean (for approximately normal distribution)'
        else:
            method = 'Median (for skewed distribution)'
    else:
        method = 'Special consideration required'

    imputation_methods.append((column, dtype, missing_count, method))

imputation_summary = pd.DataFrame(imputation_methods, columns=['Column', 'Data Type', 'Missing Values', 'Suggested Method'])

imputation_summary.sort_values(by='Missing Values', ascending=False)
# @title 填補缺失值
# Filling missing values for each column based on the suggested methods

# Defining functions for different imputation methods
def impute_mode(column):
    return column.fillna(column.mode()[0])

def impute_mean(column):
    return column.fillna(column.mean())

def impute_median(column):
    return column.fillna(column.median())

# Applying the imputation methods to each column
for column, dtype, _, method in imputation_methods:
    if method == 'Mode (for categorical data)':
        data[column] = impute_mode(data[column])
    elif method == 'Mean (for approximately normal distribution)':
        data[column] = impute_mean(data[column])
    elif method == 'Median (for skewed distribution)':
        data[column] = impute_median(data[column])

# Checking if there are any missing values left
remaining_missing_values = data.isnull().sum().sum()

remaining_missing_values, data.head()


(0,
    Unnamed: 0       ID Delivery_person_ID  Delivery_person_Age  \
 0           0  0x4607     INDORES13DEL02                  37.0   
 1           1  0xb379     BANGRES18DEL02                  34.0   
 2           2  0x5d6d     BANGRES19DEL01                  23.0   
 3           3  0x7a6a    COIMBRES13DEL02                  38.0   
 4           4  0x70a2     CHENRES12DEL01                  32.0   
 
    Delivery_person_Ratings  Restaurant_latitude  Restaurant_longitude  \
 0                      4.9            22.745049             75.892471   
 1                      4.5            12.913041             77.683237   
 2                      4.4            12.914264             77.678400   
 3                      4.7            11.003669             76.976494   
 4                      4.6            12.972793             80.249982   
 
    Delivery_location_latitude  Delivery_location_longitude  Order_Date  \
 0                   22.765049                    75.912471  2022-03-19

In [ ]:
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score, confusion_matrix, recall_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
# from lightgbm import LGBMRegressor
# !pip install catboost
# from catboost import CatBoostRegressor

def train_and_evaluate(X_train, y_train, X_test, y_test):
    models = [
        ('XGBoost', xgb.DMatrix(X_train, label=y_train), xgb.DMatrix(X_test, label=y_test), True),
        ('LightGBM', lgb.Dataset(X_train, label=y_train), lgb.Dataset(X_test, label=y_test), True),
        ('Logistic Regression', LogisticRegression(), None, True),
        ('Decision Tree', DecisionTreeClassifier(), None, False),
        ('Random Forest', RandomForestClassifier(), None, False),
        # ('SVM', SVC(probability=True), None, False),
        ('KNN', KNeighborsClassifier(n_neighbors=3), None, False),
        # ('Neural Network', MLPClassifier(hidden_layer_sizes=(100,)), None, True)

    ]

    for name, model, dtest, adjust_threshold in tqdm(models, desc="Training and evaluating models"):
        print(f"\n模型: {name}")

        print("\n")
        unique_values, counts = np.unique(y_train, return_counts=True)
        for value, count in zip(unique_values, counts):
          print(f"y_train值 {value} 出現了 {count} 次")

        unique_values, counts = np.unique(y_test, return_counts=True)
        for value, count in zip(unique_values, counts):
          print(f"y_test值 {value} 出現了 {count} 次")
        # print(X_train.describe())

        # 对于 XGBoost 和 LightGBM
        if name in ['XGBoost', 'LightGBM']:
            params = {'max_depth': 3, 'eta': 0.3, 'objective': 'binary:logistic'} if name == 'XGBoost' else {'objective': 'binary'}
            num_round = 100
            bst = xgb.train(params, model, num_round) if name == 'XGBoost' else lgb.train(params, model, num_boost_round=num_round)
            y_test_pred = bst.predict(dtest.data if name == 'LightGBM' else dtest)

        # 对于其他模型
        else:
            model.fit(X_train, y_train)
            y_test_pred = model.predict_proba(X_test)[:,1] if adjust_threshold else model.predict(X_test)

        # 评估性能
        if adjust_threshold:
            f1, conf_matrix, sensitivity, g_mean, roc_auc = evaluate_model_with_threshold(y_test, y_test_pred)
        else:
            f1, conf_matrix, sensitivity, g_mean, roc_auc = evaluate_model_without_threshold(y_test, y_test_pred)

        print(f"F1 分数: {f1}, 混淆矩阵: {conf_matrix}, Sensitivity: {sensitivity}, G-mean: {g_mean}, ROC-AUC: {roc_auc}")
        tn, fp, fn, tp = conf_matrix.ravel()
        print(f"  混淆矩阵: \n    TN: {tn}, FP: {fp}\n    FN: {fn}, TP: {tp}")


def evaluate_model_with_threshold(y_test, y_pred):
    # 确定最佳阈值
    best_threshold = 0.5
    best_f1 = 0
    for threshold in np.linspace(0, 1, 100):
        y_pred_binary = np.where(y_pred > threshold, 1, 0)
        f1 = f1_score(y_test, y_pred_binary)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    y_pred_binary = np.where(y_pred > best_threshold, 1, 0)
    return calculate_metrics(y_test, y_pred_binary)

def evaluate_model_without_threshold(y_test, y_pred):
    return calculate_metrics(y_test, y_pred)

def calculate_metrics(y_test, y_pred_binary):
    f1 = f1_score(y_test, y_pred_binary)
    conf_matrix = confusion_matrix(y_test, y_pred_binary)
    sensitivity = recall_score(y_test, y_pred_binary)
    tn, fp, fn, tp = conf_matrix.ravel()
    specificity = tn / (tn + fp)
    g_mean = np.sqrt(sensitivity * specificity)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_binary)
    roc_auc = auc(fpr, tpr)

    return f1, conf_matrix, sensitivity, g_mean, roc_auc



# 数据预处理
X = data.drop('IsGreenOrder', axis=1)
y = data['IsGreenOrder']

X_encoded = pd.get_dummies(X)
y_encoded = LabelEncoder().fit_transform(y)

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)
# len(X_train), len(X_test), len(y_train), len(y_test)

# 定义不同的数据扩充方法
methods = [
    ('原始数据', X_train, y_train),
    # ('随机过采样', *RandomOverSampler(random_state=42).fit_resample(X_train, y_train)),
    # ('随机欠采样', *RandomUnderSampler(random_state=42).fit_resample(X_train, y_train)),
    # ('SMOTE', *SMOTE(random_state=42).fit_resample(X_train, y_train)),
    # ('ADASYN', *ADASYN(random_state=42).fit_resample(X_train, y_train))
    # 更多方法...
]

# 存储结果的字典
results = {}

# 进行模型训练和评估的循环
for method_name, X_augmented, y_augmented in tqdm(methods, desc="Training Models"):
    train_and_evaluate(X_augmented, y_augmented, X_test, y_test)


Training Models:   0%|          | 0/1 [00:00<?, ?it/s]

## Target encoding

In [ ]:
encoder = MEstimateEncoder(m=1)

train= data[:len(train)]
test=data[len(train):]

train =  encoder.fit_transform(train,y)
test= encoder.transform(test)


In [ ]:
xgb_model=xgb.XGBRegressor(n_estimators=25,max_depth=7)
lgb = LGBMRegressor(num_leaves=85,max_depth=8,learning_rate=0.1)
cat= CatBoostRegressor(n_estimators=700,max_depth=8,learning_rate=0.05)

## Feature Selection

In [ ]:
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# sfs = SFS(LGBMRegressor(num_leaves=85,max_depth=8,learning_rate=0.1),
#            k_features=20,
#            forward=True,
#            floating=False,
#            verbose=2,
#            scoring='r2',
#            cv=2)

# sfs = sfs.fit(train ,y)
# selected_feat_= list(sfs.k_feature_names_)

# selected_feat_

In [ ]:
selected_feat_=['Delivery_person_Age',
 'Delivery_person_Ratings',
 'Weatherconditions',
 'Road_traffic_density',
 'Vehicle_condition',
 'multiple_deliveries',
 'Festival',
 'City',
 'distance_diff_KM',
 'n_missing',
 'Weatherconditions_missing',
 'multiple_deliveries_missing',
 'Festival_missing',
 'City_missing',
 'is_month_end',
 'is_quarter_start',
 'is_weekend',
 'feat2',
 'feat9',
 'feat12']

In [ ]:
imp_feat=selected_feat_[:12]

## Stacking

In [ ]:

#best_1

estimators = [
     ('xgb_model', xgb_model),
     ('lgb', lgb),('cat',cat)]
# best
reg = StackingRegressor(
estimators=estimators,
 final_estimator=xgb_model)

xgb_model.fit(train[imp_feat],y)
reg.fit(train,y)
lgb.fit(train[imp_feat],y)
cat.fit(train[imp_feat],y)

reg_pred=reg.predict(test)
lgb_pred=lgb.predict(test[imp_feat])
cat_pred=cat.predict(test[imp_feat])
xgb_pred=xgb_model.predict(test[imp_feat])

ensemble=reg_pred*0.2 + lgb_pred*0.45 + cat_pred*0.35  # Using these ratios after lots of experimenting




## CONCLUSION

**Finally I encourage my fellow Kagglers to use this data in order to improve their ML skills such as**
* create features on the basis of location, use different types of aggreation techinique.
* use different techinque to fill na values
* try to use different transformations
* try different feature selection techniques
* try different encoding techinqiue such as ordinal encoding etc.
* try different alogirithims